In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import cv2 
import pytesseract
import numpy as np
from pytesseract import Output
import imutils

import pandas as pd


In [3]:
#image = cv2.imread('cupom1.jpeg')

image = cv2.imread('cupom2.jpg')

img = imutils.resize(image, width=500)

# cv2.imshow('ImageWindow', img)
# cv2.waitKey(0)

rgb = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#d = pytesseract.image_to_string(rgb,output_type=Output.DICT)
threshold_img = cv2.threshold(rgb, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
threshold_img = cv2.GaussianBlur(threshold_img, (3,3), 0)
    

In [6]:
#parametros para o tesseract
custom_config = r'--oem 3 --psm 4'

details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT, config=custom_config)
text = pytesseract.image_to_string(threshold_img, output_type=Output.DICT, config=custom_config)

In [19]:
print(details.keys())
print(details['conf'])
print('-----------------------------------------')
print('-----------------------------------------')
print(details['top'])
print('-----------------------------------------')

print(details['text'])


dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])
['-1', '-1', '-1', '-1', 88, 20, 20, 89, 79, 26, '-1', 91, 81, 67, 59, '-1', 92, 91, 91, 81, 87, '-1', '-1', '-1', 95, 95, '-1', 95, '-1', '-1', '-1', 9, 0, 66, 0, 51, '-1', 46, 19, 0, 0, 0, 42, 92, '-1', 80, 89, '-1', 0, 0, 0, 0, 0, '-1', '-1', 78, 78, 85, 29, 0, 55, 0, 94, '-1', 90, 81, 74, 89, 90, 88, 58, 95, '-1', 94, 89, 80, 27, 88, '-1', '-1', 95, 68, '-1', 51, '-1', 78, 54, '-1', '-1', '-1', 95, '-1', '-1', '-1', 95, 66, 34, 36, 70, 96, '-1', 92, 92, 89, 86, '-1', 84, 92, 96, '-1', 91, 91, 92, 92, 95, '-1', '-1', 72, 83, 41, 85, 81, '-1', '-1', 44, 2, 30, '-1', 41, 43, 34, 93, 87, 50, 2, 21, 67, '-1', 62, 42, 88, '-1', 86, 43, 43, 21, 19, 0, '-1', 26, 96, '-1', 32, 49, 16, 90, '-1', 58, 54, '-1', '-1', '-1', 95]
-----------------------------------------
-----------------------------------------
[0, 35, 35, 35, 35, 36, 36, 36, 36, 36, 52, 52, 52, 53, 

In [9]:
n_boxes = len(details['text'])


for i in range(n_boxes):
    if int(details['conf'][i]) > 60:
        (x, y, w, h) = (details['left'][i], details['top'][i], details['width'][i], details['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# cv2.imshow('img', img)
# cv2.waitKey(0)

In [10]:
parse_text = []

word_list = []

last_word = []

for word in details['text']:
    if word!='':
        word_list.append(word)
        last_word = word
    if(last_word!='' and word =='') or (word==details['text'][-1]):
        parse_text.append(word_list)
        
        word_list = []
    

In [21]:
print(parse_text)

[[], [], [], [], ['CARREFUUR', 'CUMERC', 'EO', 'E', 'INDUSTRIA', 'LIOR'], ['LAGO', 'SUL', 'BRASIL', 'IA~DF'], ['SHIS', 'QI', '25', 'LOIE', 'G'], [], [], [' ', ' '], [' '], [], [], ['(NPD:', '45.543.915/006-1', 'TE:', '07.312.', '810/006-86_'], ['2', '50', 'CF)', 'eet', 'ia.', 'coo', '4'], ['JPOM', 'FISC'], ['(', '16)', 'O2O.UN.', 'YL', 'rickemon'], [], ['001', '00515647', 'CREME', 'LETTE', 'W', '1', '117,00', '1,65)'], ['002', '0051564?', 'CREME', 'LETTE', 'N', '2', 'T1?,00%', '1,65)'], ['003', '00515647', '(RENE', 'LETTE', 'N'], [], ['TOTAL', 'RS'], ['DINHEIROv'], ['TROCO', 'R$'], [], [], [' '], [], [], ['ORT:', '901262113', 'POV:', '7?', 'CUPON:', '37329'], ['PROCON', '-', 'FONE:', '3212-2500'], ['SAC_CARREFOUR-0800', '727', '3090'], ['TELEFONE', ':', '3367-7039', '-', '3367-3734'], [], ['fiplicativo', 'Calypso', '~', 'Ver.:', 'CA.15.cOL'], [], ['HDS:', 'dhfGe92a0t698e435', '1da9825b9000220'], ['OER', '267A7', '1FO1B', '783139', '48', 'S4AUBF', '0AB58', '208C7', '6C4'], ['DARUNR', 'A